In [5]:
# Define some helper functions
def readCSV(fname, removeHeader=False, separator=','):
    print("Loading file ", fname, "...")
    rdd = sc.textFile(fname)
    if removeHeader:
        firstline = rdd.first()
        rdd = rdd.filter(lambda x: x != firstline)
    return rdd.map(lambda x: x.split(separator))

# Load the movies and ratings database
movies = readCSV("./movies.csv", removeHeader=True)
ratings = readCSV("./ratings_train.csv", removeHeader=True)

# Print the first entries to debug whether the data is loaded correctly
print("First movie:", movies.first())
print("First rating:", ratings.first())

# Parse the movie genres
# [id, name, genres[]]
movies = movies.map(lambda x: [x[0], x[1], x[2].split('|')])
print("First movie, processed:", movies.first())

# Parse the rating data
# [user_id, movie_id, rating, timestamp]
ratings = ratings.map(lambda x: x[0].split('::'))
print("First rating, processed:", ratings.first())

# TEST: Calculate rating averages
print("Calculating ratings average...")
user_ratings = ratings.map(lambda x: float(x[2]))
print("Avg rating:", user_ratings.sum()/user_ratings.count())

Loading file  ./movies.csv ...
Loading file  ./ratings_train.csv ...
First movie: ['1', 'Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy']
First rating: ['11973::11::3.0::943354625']
First movie, processed: ['1', 'Toy Story (1995)', ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']]
First rating, processed: ['11973', '11', '3.0', '943354625']
Calculating ratings average...
Avg rating: 3.519642305478569
